### 데이콘에서 제공한 변수별 제한 수치
- 시간당 내부 평균온도: 4도~40도  
- 시간당 내부 평균습도: 0% ~ 100%  
- 시간당 내부 평균 co2 농도 : 0ppm ~ 1200 ppm  
- 시간당 평균 EC : 0 ~ 8  
- 시간당 분무량 : 0 ~ 3000 / 일간 누적 분무량 0 ~ 72,000  
- 시간당 백색광량 : 0 ~ 120,000 / 일간 누적 백색광량 0 ~ 2,880,000  
- 시간당 적색광량 : 0 ~ 120,000 / 일간 누적 적색광량 0 ~ 2,880,000   
- 시간당 청색광량 : 0 ~ 120,000 / 일간 누적 청색광량 0 ~ 2,880,000  
- 시간당 총광량 : 0 ~ 120,000 / 일간 누적 총광량 0 ~ 2,880,000  

# Data Load

In [1]:
import pandas as pd

In [2]:
from modules.data_store import data_store

There is data stored
Last data fetch completed


In [3]:
data_store = data_store()

Load the saved pickle file


In [4]:
trial = data_store.return_last_trial()
df = data_store.dict_df[trial]
df.columns

Index(['DAT', 'predicted_weight_g', 'train/test', 'file_num', 'obs_time',
       '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량',
       '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량',
       '일간누적총광량', 'sum', 'total_sum', 'weight', 'target'],
      dtype='object')

In [5]:
df[df['train/test']=='CASE'].describe()

,DAT,predicted_weight_g,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
count,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,1.881600e+04,18816.000000,18816.000000
mean,14.500000,34.919681,26.027719,63.935090,498.860695,1.507566,523.627918,6365.916673,6315.562930,72910.240149,1571.862912,18201.358998,1241.905981,14139.184959,9129.331824,105250.784106,19372.622634,4.649429e+05,0.041667,1.454987
std,8.077962,41.580313,3.948988,11.657905,120.421248,1.042010,544.323281,6903.680044,8188.067575,86040.447105,2937.252667,34386.384982,2474.426437,27743.227265,9175.427411,97397.224913,18416.095388,1.578185e+05,0.039194,2.277581
min,1.000000,0.167719,0.000000,0.000000,60.400000,0.000000,-653.260000,0.000000,-138371.246100,0.000000,-8793.607200,0.000000,-18570.857500,0.000000,-165735.710800,0.000000,-330190.428957,3.975293e+04,-2.121948,-4.470883
25%,7.750000,4.869563,23.971250,57.494583,420.245833,0.929581,0.000000,1234.280000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7280.962850,876.121634,3.967611e+05,0.002430,0.040572
50%,14.500000,16.118144,26.291667,62.118333,473.983333,1.308517,252.000000,3360.485000,0.000000,23205.750000,0.000000,0.000000,0.000000,0.000000,9242.350000,92172.360000,19796.105613,4.422953e+05,0.047776,0.298112
75%,21.250000,51.272294,28.106667,73.157166,551.105952,2.291322,769.000000,9990.570000,18236.625400,145422.700000,1494.698100,17529.296400,1306.461000,18484.700000,18255.190000,177491.499450,37204.070917,5.356698e+05,0.075820,1.809942
max,28.000000,164.150300,42.643334,93.350002,1714.850000,5.462324,3144.130000,42855.230000,18564.600000,292014.969800,9928.800000,200561.760000,9399.000000,190016.450000,37892.400000,559488.881000,76641.128238,1.136716e+06,0.436592,12.043547


- obs_time 통일  
- 내부온도관측치 0 처리  
- 내부습도관측치 0 처리    
- 시간당백색광량, 시간당적색광량, 시간당청색광량 마이너스값 조절  
- 모든 누적광량 컬럼 cumsum 재 계산  
- 적색광,청색광 값이 0인 부분이 많은데 채워줘야하는지?   

null값 처리할건 처리하고 null값 채워주는 모듈 사용한뒤 제출해보기  
<br>

이상치 모듈 한번더 실행후 EBM해보기


## obs_time 통일

In [6]:
def get_obs_time(x):
    time = x.split(':')
    if int(time[1]) == 0:
        return int(time[0])
    else :
        return int(time[0])+1
    
df['obs_hour'] = df['obs_time'].apply(get_obs_time)
df['obs_hour'].value_counts()

15    924
14    924
16    924
1     924
17    924
2     924
18    924
3     924
19    924
4     924
20    924
5     924
21    924
6     924
22    924
7     924
23    924
8     924
9     924
10    924
11    924
12    924
13    924
0     924
Name: obs_hour, dtype: int64

## 내부온도관측치 0인 값 처리

In [7]:
df[df['내부온도관측치']==0]

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour
16143,1,1.656917,CASE,04,15:00,0.0,0.0,560.254237,1.436618,1550.86,...,10160.472,3565.354,21154.016,23634.2048,182458.1789,49380.960456,497502.195384,0.099258,0.164462,15


In [8]:
def load_data(df,DAT,train_test,file_num):
    return df[(df['DAT']==DAT)&(df['train/test']==train_test)&(df['file_num']==file_num)]
load_data(df,1,'CASE','04').drop(['DAT','train/test','file_num','sum','total_sum'],axis=1)

,predicted_weight_g,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,weight,target,obs_hour
16128,1.656917,00:00,16.276667,58.665000,602.633333,0.003096,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001362,0.002257,0
16129,1.656917,01:00,16.370000,58.231667,593.616667,0.003903,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001343,0.002225,1
16130,1.656917,02:00,16.286667,58.146667,584.783333,0.004276,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001325,0.002196,2
16131,1.656917,03:00,16.368333,57.860000,578.350000,0.004626,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001312,0.002173,3
16132,1.656917,04:00,16.371667,57.481667,569.966667,0.004987,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001294,0.002144,4
16133,1.656917,05:00,16.330000,57.521667,563.883333,0.004963,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.001282,0.002124,5
16134,1.656917,06:00,16.686667,56.460000,560.100000,0.005228,0.00,0.00,4201.7878,4201.7878,188.6472,188.6472,178.5810,178.5810,4569.0160,4569.0160,0.019641,0.032543,6
16135,1.656917,07:00,17.503333,53.740000,556.100000,0.005300,0.00,0.00,13289.1595,17490.9473,671.8488,860.4960,635.9990,814.5800,14597.0073,19166.0233,0.059942,0.099319,7
16136,1.656917,08:00,19.920000,49.250526,554.583333,0.008625,0.00,0.00,18242.8136,35733.7609,973.0224,1833.5184,1315.8600,2130.4400,20531.6960,39697.7193,0.083793,0.138838,8
16137,1.656917,09:00,24.740000,41.851069,548.750000,0.008408,0.00,0.00,18255.1900,53988.9509,976.3320,2809.8504,2232.2625,4362.7025,21463.7845,61161.5038,0.087523,0.145019,9


- 온도가 0인값이 있고, 그전 값 역시 6도로 이상치처럼 보인다.  
- 온도가 6일때 분무량은 높고,광량은 낮은것으로 확인된다 하지만 온도가 6인것이 분무량과 광량과 상관이 없다. -> 이상치 처리 해준다.

내부온도관측치가 이전 시간의 온도 값과 차이가 상대적으로 높은것이 또 있는지 확인해본다.

In [9]:
df['diff_내부온도관측치'] = df.groupby(['DAT','train/test','file_num']).shift(1).fillna(0)['내부온도관측치'] - df['내부온도관측치']
df

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour,diff_내부온도관측치
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.000000,61.7,...,0.0,0.0,0.0000,0.000,629.156667,381790.957381,0.001648,0.003161,0,-27.680000
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.000000,61.7,...,0.0,0.0,0.0000,0.000,627.106666,381790.957381,0.001643,0.003151,1,-0.090000
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.000000,61.7,...,0.0,0.0,0.0000,0.000,628.830000,381790.957381,0.001647,0.003159,2,0.003333
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.000000,61.7,...,0.0,0.0,0.0000,0.000,631.551667,381790.957381,0.001654,0.003173,3,-0.130000
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.000000,61.7,...,0.0,0.0,0.0000,0.000,634.715000,381790.957381,0.001662,0.003189,4,-0.025000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22171,28,0.000000,TEST,02,19:00,26.960000,56.620000,542.616667,1.067239,769.0,...,0.0,0.0,12.3764,209656.216,1421.016706,446639.947445,0.003182,0.000000,19,2.800000
22172,28,0.000000,TEST,02,20:00,26.211667,57.241667,540.583333,1.074064,0.0,...,0.0,0.0,0.0000,209656.216,625.110731,446639.947445,0.001400,0.000000,20,0.748333
22173,28,0.000000,TEST,02,21:00,25.368333,56.310000,530.800000,1.078533,769.0,...,0.0,0.0,0.0000,209656.216,1382.556867,446639.947445,0.003095,0.000000,21,0.843334
22174,28,0.000000,TEST,02,22:00,25.256667,57.343333,540.466667,1.078533,0.0,...,0.0,0.0,0.0000,209656.216,624.145200,446639.947445,0.001397,0.000000,22,0.111667


In [10]:
df[(df['train/test']=='TEST')&(df['obs_hour']!=0)][['diff_내부온도관측치']].describe()

,diff_내부온도관측치
count,3220.000000
mean,-0.003731
std,0.926230
min,-4.088334
25%,-0.238333
50%,-0.010000
75%,0.179999
max,5.248333


TEST set에서는 없다

In [11]:
df[(df['train/test']=='CASE')&(df['obs_hour']!=0)][['diff_내부온도관측치']].describe()

,diff_내부온도관측치
count,18032.000000
mean,-0.029332
std,1.009536
min,-16.516215
25%,-0.251667
50%,0.002952
75%,0.231667
max,19.168948


train set에는 존재

In [12]:
df[df['diff_내부온도관측치']>=6]

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour,diff_내부온도관측치
8754,1,0.258003,CASE,15,18:00:00.,30.895000,49.592813,516.066667,0.369954,0.00,...,3696.940,55391.4400,9554.9320,143780.8520,19706.788434,410906.797631,0.047959,0.012374,18,8.263824
11481,3,1.333024,CASE,02,09:00:00.,15.745642,23.504397,609.650000,2.977497,123.40,...,4621.175,13663.0130,9502.8350,28096.1786,19780.947536,246001.776721,0.080410,0.107188,9,8.692691
11771,15,10.216160,CASE,02,11:00:00.,18.435500,50.093473,621.983333,3.720377,123.40,...,4621.175,22878.7325,9502.8350,47047.0865,19823.302684,246770.636512,0.080331,0.820673,11,6.075500
16142,1,1.656917,CASE,04,14:00,6.921053,9.639473,564.105263,1.471572,1550.86,...,1105.949,17588.6620,7525.8586,158823.9741,17184.714561,497502.195384,0.034542,0.057233,14,19.168948
16143,1,1.656917,CASE,04,15:00,0.000000,0.000000,560.254237,1.436618,1550.86,...,3565.354,21154.0160,23634.2048,182458.1789,49380.960456,497502.195384,0.099258,0.164462,15,6.921053


- 확인 결과, 온도가 0,6 인것만 이상치로 판단됨.

In [13]:
def change_temperature(x):
    if x < 10 :
        return None
    else :
        return x
    
df['change_내부온도관측치'] = df['내부온도관측치'].apply(change_temperature)
df = df.fillna(method = 'ffill')


## 시간당백색광량, 시간당적색광량, 시간당청색광량 마이너스값 이전값 대치  


In [14]:
df[df['시간당백색광량']<0]

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour,diff_내부온도관측치,change_내부온도관측치
14799,1,2.106971,CASE,06,15:00:00.,27.384,74.1,410.12,0.388644,769.0,...,1372.254,-165735.7108,9177.0808,-330190.428957,155607.198641,-2.121948,-4.470883,15,0.121,27.384


In [15]:
def change_light(x):
    if x < 0 :
        return None
    else :
        return x
    
df['change_시간당백색광량'] = df['시간당백색광량'].apply(change_light)
df['change_시간당청색광량'] = df['시간당청색광량'].apply(change_light)
df['change_시간당적색광량'] = df['시간당적색광량'].apply(change_light)
df = df.fillna(method = 'ffill')



In [16]:
df[df['시간당백색광량']<0]

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,sum,total_sum,weight,target,obs_hour,diff_내부온도관측치,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량
14799,1,2.106971,CASE,06,15:00:00.,27.384,74.1,410.12,0.388644,769.0,...,-330190.428957,155607.198641,-2.121948,-4.470883,15,0.121,27.384,18255.19,3419.6695,1659.7644


## 모든 누적광량 컬럼 cumsum 재 계산

In [17]:
df.columns

Index(['DAT', 'predicted_weight_g', 'train/test', 'file_num', 'obs_time',
       '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량',
       '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량',
       '일간누적총광량', 'sum', 'total_sum', 'weight', 'target', 'obs_hour',
       'diff_내부온도관측치', 'change_내부온도관측치', 'change_시간당백색광량', 'change_시간당청색광량',
       'change_시간당적색광량'],
      dtype='object')

In [18]:
df['change_일간누적백색광량'] = df.groupby(['DAT','train/test','file_num'])['change_시간당백색광량'].cumsum()
df['change_일간누적청색광량'] = df.groupby(['DAT','train/test','file_num'])['change_시간당청색광량'].cumsum()
df['change_일간누적적색광량'] = df.groupby(['DAT','train/test','file_num'])['change_시간당적색광량'].cumsum()

In [19]:
df['change_시간당총광량'] = df['change_시간당백색광량'] + df['change_시간당청색광량'] + df['change_시간당적색광량']

In [20]:
df['change_일간누적총광량'] = df.groupby(['DAT','train/test','file_num'])['change_시간당총광량'].cumsum()
df.head(1)

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,diff_내부온도관측치,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량
0,1,1.918119,CASE,08,00:00,27.68,73.026667,466.75,0.0,61.7,...,-27.68,27.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## sum, total_sum, weight, target도  재설정

In [21]:
sum_cols = [ 'change_내부온도관측치',
 '내부습도관측치',
 'co2관측치',
 'ec관측치',
 '시간당분무량',
 'change_시간당백색광량',
 'change_시간당적색광량',
 'change_시간당청색광량',
 'change_시간당총광량']

In [22]:
df['sum'] = df[sum_cols].sum(axis=1)
df.head(1)

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,diff_내부온도관측치,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량
0,1,1.918119,CASE,08,00:00,27.68,73.026667,466.75,0.0,61.7,...,-27.68,27.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
total_sum_df = df[['DAT','train/test','file_num','sum']].groupby(['DAT','train/test','file_num']).sum().reset_index()
total_sum_df.columns = ['DAT','train/test','file_num','total_sum']

base_df = pd.merge(df.drop('total_sum',axis=1),total_sum_df,how='inner',on=['DAT','train/test','file_num'])
base_df['weight'] = base_df['sum'] / base_df['total_sum']
base_df['target'] = base_df['predicted_weight_g'] * base_df['weight']
base_df

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량,total_sum
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.000000,61.7,...,27.680000,0.0000,0.0,0.0,0.000,0.0,0.0,0.0000,0.000,381790.957381
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.000000,61.7,...,27.770000,0.0000,0.0,0.0,0.000,0.0,0.0,0.0000,0.000,381790.957381
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.000000,61.7,...,27.766666,0.0000,0.0,0.0,0.000,0.0,0.0,0.0000,0.000,381790.957381
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.000000,61.7,...,27.896666,0.0000,0.0,0.0,0.000,0.0,0.0,0.0000,0.000,381790.957381
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.000000,61.7,...,27.921666,0.0000,0.0,0.0,0.000,0.0,0.0,0.0000,0.000,381790.957381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22171,28,0.000000,TEST,02,19:00,26.960000,56.620000,542.616667,1.067239,769.0,...,26.960000,12.3764,0.0,0.0,209656.216,0.0,0.0,12.3764,209656.216,446639.947445
22172,28,0.000000,TEST,02,20:00,26.211667,57.241667,540.583333,1.074064,0.0,...,26.211667,0.0000,0.0,0.0,209656.216,0.0,0.0,0.0000,209656.216,446639.947445
22173,28,0.000000,TEST,02,21:00,25.368333,56.310000,530.800000,1.078533,769.0,...,25.368333,0.0000,0.0,0.0,209656.216,0.0,0.0,0.0000,209656.216,446639.947445
22174,28,0.000000,TEST,02,22:00,25.256667,57.343333,540.466667,1.078533,0.0,...,25.256667,0.0000,0.0,0.0,209656.216,0.0,0.0,0.0000,209656.216,446639.947445


In [25]:
load_data(base_df,1,'CASE','08')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량,total_sum
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.0,61.7,...,27.680000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.0,61.7,...,27.770000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.0,61.7,...,27.766666,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.0,61.7,...,27.896666,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.0,61.7,...,27.921666,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
5,1,1.918119,CASE,08,05:00,27.828333,73.635000,475.033333,0.0,61.7,...,27.828333,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,381790.957381
6,1,1.918119,CASE,08,06:00,28.051667,73.621668,485.683333,0.0,123.4,...,28.051667,4269.8580,0.0,0.0,4269.8580,0.0,0.0,4269.8580,4269.8580,381790.957381
7,1,1.918119,CASE,08,07:00,29.698333,72.093334,486.900000,0.0,123.4,...,29.698333,13326.2887,0.0,0.0,17596.1467,0.0,0.0,13326.2887,17596.1467,381790.957381
8,1,1.918119,CASE,08,08:00,31.133333,70.118334,481.516667,0.0,123.4,...,31.133333,18249.0018,0.0,0.0,35845.1485,0.0,0.0,18249.0018,35845.1485,381790.957381
9,1,1.918119,CASE,08,09:00,30.758333,69.293332,506.300000,0.0,123.4,...,30.758333,18255.1900,0.0,0.0,54100.3385,0.0,0.0,18255.1900,54100.3385,381790.957381


## data 저장후 성능 확인해 보기

In [26]:
data_store.add_df(all_df = base_df)
data_store.save_pkl()

## EBM

In [27]:
base_df.columns

Index(['DAT', 'predicted_weight_g', 'train/test', 'file_num', 'obs_time',
       '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량',
       '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량',
       '일간누적총광량', 'sum', 'weight', 'target', 'obs_hour', 'diff_내부온도관측치',
       'change_내부온도관측치', 'change_시간당백색광량', 'change_시간당청색광량', 'change_시간당적색광량',
       'change_일간누적백색광량', 'change_일간누적청색광량', 'change_일간누적적색광량',
       'change_시간당총광량', 'change_일간누적총광량', 'total_sum'],
      dtype='object')

In [28]:
from modules import config
config.input_cols

['DAT',
 'obs_hour',
 'change_내부온도관측치',
 '내부습도관측치',
 'co2관측치',
 'ec관측치',
 '시간당분무량',
 '일간누적분무량',
 'change_시간당백색광량',
 'change_일간누적백색광량',
 'change_시간당적색광량',
 'change_일간누적적색광량',
 'change_시간당청색광량',
 'change_일간누적청색광량',
 'change_시간당총광량',
 'change_일간누적총광량']

In [29]:
train_df = base_df[base_df['train/test']=='CASE']
test_df = base_df[base_df['train/test']=='TEST']

In [30]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train_df[config.input_cols]
y_train = train_df['target']

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_hour', 'change_내부온도관측치',
                                            '내부습도관측치', 'co2관측치', 'ec관측치',
                                            '시간당분무량', '일간누적분무량',
                                            'change_시간당백색광량', 'change_일간누적백색광량',
                                            'change_시간당적색광량', 'change_일간누적적색광량',
                                            'change_시간당청색광량', 'change_일간누적청색광량',
                                            'change_시간당총광량', 'change_일간누적총광량',
                                            'DAT x change_시간당총광량',
                                            'DAT x change_시간당백색광량',
                                            'DAT x change_일간누적총광량',
                                            'DAT x obs_hour', 'DAT x 시간당분무량...
                             feature_types=['continuous', 'continuous',
                                            'continuous', 'continuous',
                                

In [33]:
from interpret import show
from interpret.data import Marginal

ebm_global = ebm.explain_global()
show(ebm_global)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/140573338951504/ -->

In [39]:

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

<!-- http://127.0.0.1:7001/140573381979472/ -->

In [34]:
cols = config.input_cols

def get_eachCols_EBM_score(ebm_global, cols):
    EBM_cols_dict = dict()
    for i in range(len(cols)):
        EBM_cols_dict[cols[i]] = {'values':ebm_global.data(i)['names'],'scores':list(ebm_global.data(i)['scores'])}
    return EBM_cols_dict
EBM_cols_dict = get_eachCols_EBM_score(ebm_global, cols)
# EBM_cols_dict['DAT']

In [35]:
import numpy as np
def get_EBM_MinMax_scorePoint(df, EBM_cols_dict):
    EBM_Score_Min_max_df = dict()
    for k,v in EBM_cols_dict.items():
        EBM_Score_Min_max_df[k] = {'Unique':len(set(df[k])),'Max_Point':v['values'][v['scores'].index(max(v['scores']))],'Min_Point':v['values'][v['scores'].index(min(v['scores']))] ,'Max_Score' : int(max(v['scores'])), 'Min_Score' : int(min(v['scores'])), 'Midean' : int(np.median(v['scores']))}
    EBM_Score_Min_max_df = pd.DataFrame(EBM_Score_Min_max_df).transpose()
    EBM_Score_Min_max_df['Range_Score'] = EBM_Score_Min_max_df['Max_Score'] - EBM_Score_Min_max_df['Min_Score']
    return EBM_Score_Min_max_df.sort_values('Range_Score',ascending=False)
get_EBM_MinMax_scorePoint(train_df,EBM_cols_dict)

,Unique,Max_Point,Min_Point,Max_Score,Min_Score,Midean,Range_Score
DAT,28.0,27.500000,3.500000,2.0,0.0,0.0,2.0
co2관측치,14246.0,60.400000,623.291667,2.0,0.0,0.0,2.0
change_시간당총광량,2518.0,13905.061000,37260.860000,0.0,-2.0,0.0,2.0
ec관측치,16306.0,3.657944,0.241450,0.0,-1.0,0.0,1.0
시간당분무량,1869.0,2259.725000,123.400000,1.0,0.0,0.0,1.0
change_시간당백색광량,1143.0,12775.538900,8533.527800,0.0,-1.0,0.0,1.0
change_시간당적색광량,887.0,9597.840000,8687.700000,1.0,0.0,0.0,1.0
change_일간누적적색광량,2884.0,154.723800,20425.196400,0.0,-1.0,0.0,1.0
change_시간당청색광량,959.0,9164.025000,4621.175000,1.0,0.0,0.0,1.0
change_일간누적청색광량,2863.0,151245.575000,101872.628000,1.0,0.0,0.0,1.0


In [45]:
base_df[base_df['change_일간누적총광량']>=500000][['total_sum','DAT','target','file_num']]

,total_sum,DAT,target,file_num
1387,1.134282e+06,2,0.100724,21
1388,1.134282e+06,2,0.100739,21
1389,1.134282e+06,2,0.004976,21
1390,1.134282e+06,2,0.001020,21
1391,1.134282e+06,2,0.001195,21
...,...,...,...,...
2011,1.133306e+06,28,1.521785,21
2012,1.133306e+06,28,1.519065,21
2013,1.133306e+06,28,0.080374,21
2014,1.133306e+06,28,0.015993,21


# test predict

In [36]:
X_test = test_df[config.input_cols]
y_test = test_df['target']


test_df['predicted_weight_g'] = ebm.predict(X_test)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
base_rslt_df = test_df[['DAT','file_num','predicted_weight_g']].groupby(['DAT','file_num']).sum().reset_index()
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

base_rslt_df['predicted_weight_g'] = base_rslt_df['predicted_weight_g'].apply(predict_rslt_weight)
# base_rslt_df = base_rslt_df.set_index('DAT')
base_rslt_df

,DAT,file_num,predicted_weight_g
0,1,01,7.783934
1,1,02,0.000000
2,1,03,0.000000
3,1,04,1.191697
4,1,05,1.468794
...,...,...,...
135,28,01,129.322009
136,28,02,79.277571
137,28,03,18.364355
138,28,04,135.382398


In [38]:
file_num_list = set(base_rslt_df['file_num'])
test_target_list = [] 
file_path = './open/sample_submission/'
for file_num in file_num_list:
    test_target_file = file_path+'TEST_'+file_num+'.csv'
    print(test_target_file)
    test_target_list.append(test_target_file)
    base_rslt_df[base_rslt_df['file_num']==file_num][['DAT','predicted_weight_g']].to_csv(test_target_file,index=False)
    

./open/sample_submission/TEST_02.csv
./open/sample_submission/TEST_01.csv
./open/sample_submission/TEST_05.csv
./open/sample_submission/TEST_04.csv
./open/sample_submission/TEST_03.csv


- RMSE 값 7.45로 내려갔다.
- 내생각엔 target값을 구하는 과정에서 문제가 있는거 같음.
- 변수간 상대적 크기가 다른데 target값을 구하는 변수들의 합으로 구하는게 오류였음.
- normalize 한뒤 target 값을 다시 구하자.

In [55]:
load_data(base_df,28,'CASE','04')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량,total_sum
16776,28,128.563772,CASE,04,00:00,16.993333,72.760000,592.583333,1.125295,1113.62,...,16.993333,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16777,28,128.563772,CASE,04,01:00,16.743333,72.520000,592.750000,1.127111,0.00,...,16.743333,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16778,28,128.563772,CASE,04,02:00,16.593334,72.501667,596.083333,1.124612,1499.42,...,16.593334,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16779,28,128.563772,CASE,04,03:00,16.348333,72.763334,589.266667,1.126557,0.00,...,16.348333,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16780,28,128.563772,CASE,04,04:00,16.276666,73.138333,586.916667,1.124004,1499.42,...,16.276666,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16781,28,128.563772,CASE,04,05:00,16.261667,74.570000,587.066667,1.125822,0.00,...,16.261667,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,546507.338004
16782,28,128.563772,CASE,04,06:00,16.701667,76.555000,550.100000,1.123095,1525.14,...,16.701667,4235.8229,181.7140,191.9568,4235.8229,181.7140,191.9568,4609.4937,4609.4937,546507.338004
16783,28,128.563772,CASE,04,07:00,17.540000,76.681667,433.150000,1.120724,0.00,...,17.540000,13412.9235,640.6985,676.8132,17648.7464,822.4125,868.7700,14730.4352,19339.9289,546507.338004
16784,28,128.563772,CASE,04,08:00,20.173333,74.562000,400.301724,1.117478,1525.14,...,20.173333,18249.0018,1323.6925,974.6772,35897.7482,2146.1050,1843.4472,20547.3715,39887.3004,546507.338004
16785,28,128.563772,CASE,04,09:00,24.401667,77.510000,421.658333,1.130592,1499.42,...,24.401667,18255.1900,2241.6615,976.3320,54152.9382,4387.7665,2819.7792,21473.1835,61360.4839,546507.338004


In [54]:
load_data(base_df,28,'CASE','21')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,change_내부온도관측치,change_시간당백색광량,change_시간당청색광량,change_시간당적색광량,change_일간누적백색광량,change_일간누적청색광량,change_일간누적적색광량,change_시간당총광량,change_일간누적총광량,total_sum
1992,28,22.931393,CASE,21,00:00,23.453333,82.711667,563.816667,0.898359,126.00,...,23.453333,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1993,28,22.931393,CASE,21,01:00,23.493333,83.336667,556.450000,0.902190,0.00,...,23.493333,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1994,28,22.931393,CASE,21,02:00,23.418333,82.913333,549.233333,0.903224,119.57,...,23.418333,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1995,28,22.931393,CASE,21,03:00,23.426666,83.378334,552.866667,0.903693,0.00,...,23.426666,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1996,28,22.931393,CASE,21,04:00,23.421666,83.480000,552.083333,0.903968,0.00,...,23.421666,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1997,28,22.931393,CASE,21,05:00,23.430000,83.145000,547.400000,0.903580,0.00,...,23.430000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.133306e+06
1998,28,22.931393,CASE,21,06:00,27.175000,80.053334,426.683333,0.896274,126.00,...,27.175000,17636.3700,9085.700,9432.3600,17636.3700,9085.700,9432.3600,36154.4300,36154.4300,1.133306e+06
1999,28,22.931393,CASE,21,07:00,28.713333,80.495000,389.583333,0.886479,0.00,...,28.713333,18255.1900,9242.350,9763.3200,35891.5600,18328.050,19195.6800,37260.8600,73415.2900,1.133306e+06
2000,28,22.931393,CASE,21,08:00,29.225000,80.338333,389.949153,0.882535,126.00,...,29.225000,18255.1900,9242.350,9763.3200,54146.7500,27570.400,28959.0000,37260.8600,110676.1500,1.133306e+06
2001,28,22.931393,CASE,21,09:00,28.691667,80.728333,393.333333,0.880418,0.00,...,28.691667,18255.1900,9242.350,9763.3200,72401.9400,36812.750,38722.3200,37260.8600,147937.0100,1.133306e+06
